<img src="../figs/eh_logo.png" style="width: 200px;">

# EmptyHeaded Query Language

This tutorial provides the basic information concerning our querylanguage
The EmptyHeaded query language is standard datalog with aggregations and recursion added. We go over each of the following portions of the query language.

1. [Joins](#join)
2. [Projections](#projection)
3. [Selections](#selection)
4. [Aggregations](#agg)
5. [Recursion](#recursion)

First lets discuss what we support in conjunction with Pandas DataFrames though.

## Spinning up the engine

The steps here are pulled directly from our getting started tutorial. 

Accepted data types. The accepted datatypes when transferring Pandas DataFrames into the EmptyHeaded engine are:

- np.int32,
- np.int64,
- np.uint32,
- np.uint64,
- np.float32,
- np.float64

In [1]:
from emptyheaded import *
start()
eh = os.path.expandvars("$EMPTYHEADED_HOME")
df_graph = pd.read_csv(eh+'/test/graph/data/facebook_duplicated.tsv',\
  sep='\t',\
  names=["0","1"],\
  dtype={"0":np.uint32,"1":np.uint32})
edge = Relation(
  name="Edge",
  dataframe=df_graph)
db = Database.create(
  Config(num_threads=4),
  eh+"/docs/dbql",
  [edge])
db.build()

cd /Users/caberger/Documents/Research/code/EmptyHeaded/storage_engine/build && cmake -DNUM_THREADS=4 .. && make && cd - > /dev/null


<a id='join'></a>
## Joins

Joins can be expressed in EmptyHeaded as in standard datalog. Simply specify the relation and the attributes that you are joining. EmptyHeaded currently supports equality joins and relations with the same attribute are joined on those matching attributes. For example consider the triangle query which performs joins over the *Edge* relation. 

In [2]:
db.eval("Triangle(a,b,c) :- Edge(a,b),Edge(b,c),Edge(a,c).")

<a id='projection'></a>
## Projections

Projections are easily expressed by simply removing the attributes that are projected from the head of the rule. For example consider the triangle query where we just find all edges that participate in a triangle. We can project away the *c* attribute. The query below does this.

In [3]:
db.eval("Triangle(a,b) :- Edge(a,b),Edge(b,c),Edge(a,c).")

<a id='selection'></a>
## Selections

EmptyHeaded currently supports equality selections by simply adding an additional condition to the body of the datalog rule. For example consider the query where we find all 4-cliques connected to a specified node. Note: equality selections currently must be projected away(ie not appear in the head).

In [9]:
db.eval("""FliqueSel(a,b,c,d) :- x=0,
 Edge(a,b),Edge(b,c),Edge(a,c),
 Edge(a,d),Edge(b,d),Edge(c,d),Edge(a,x).""")

<a id='agg'></a>
## Aggregations

### Overview
We add basic aggregations to EmptyHeaded following the work of [Puttagunta et al](http://arxiv.org/abs/1508.07532). The key optimization that we use in EmptyHeaded which other worst-case optimal join processing engines do not use is the ability perform aggregations. We use *generalized hypertree decompositions* (GHDs) as our query plans in EmptyHeaded (thus our our equivalent of relational algebra). This allows us to treat aggregations as tropical semirings that support standard aggregation operations like $\sum,\times,\max,\min$.  

### Annotations

These annotations support aggregations from
any semiring (a generalization of natural numbers equipped with a
notion of addition and multiplication). This enables EmptyHeaded to support
classic aggregations such as $SUM$, $MIN$, or $COUNT$, but also
enables \EH to support more complicated computations such as gradient
descent. To specify the annotation, one uses a semicolon in the head
of the rule, e.g. $q(x, y; z:int)$ specifies that each $x,y$ pair will
be associated with an integer value with alias $z$ similar to a GROUP
BY in SQL. In addition, the user expresses the aggregation operation
in the body of the rule. The user can specify an initialization value
as any expression over the tuples' values and constants, while common
aggregates have default values. A typical
count query is shown next.

### Composing the query

Now that the database is loaded lets write a simple aggregation query. We call this query the Barbell query or $B_{3,1}$ as it finds all 3-cliques connected by a path of length one. The syntax below shows that the annotation is first specified in the head (e.g. `w`) with the aggregation appearing in the body of the rule (e.g. `w:long<-[COUNT(*)]`). Lets find out how many barbells are in our facebook dataset.


In [ ]:
db.eval("""
BarbellAgg(;w) :- w:long<-[COUNT(*)],Edge(a,b),Edge(b,c),Edge(a,c),
  Edge(a,x),Edge(x,y),Edge(y,z),Edge(x,z).
""")

<a id='recursion'></a>

## Recursion

We have support for simple (1 base case, 1 recursive statement) recursive queries in the current release of EmptyHeaded and plan to expand this support in the future. In this tutorial we demonstrate the computation of PageRank using power iteration.  


In [14]:
db.eval("""N(;w) :- Edge(x,y),w:long<-[SUM(x;1)].
PageRank(x;y) :- Edge(x,z),y:float<-[(1.0/N)].
PageRank(x;y)*[i=5]:-Edge(x,z),PageRank(z),InvDegree(z),y:float <- [0.15+0.85*SUM(z;1.0)].""")

ERROR: An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 90))



java.lang.IllegalArgumentExceptionPyRaisable: java.lang.IllegalArgumentException: Schema not found.

This is an example of the syntax used to express the PageRank query in EmptyHeaded. The first line specifies that we aggregate over all the edges in the graph and count the number of source nodes assuming our *Edge* relation is two-attribute relation filled with *(src,dst)* pairs. For an undirected graph this simply counts the number of nodes in the graph and assigns it to the relation *N* which is really just a scalar integer. By definition the *COUNT* aggregation and by default the*SUM* use an initialization value of *1* if the relation is not annotated. The second line of the query defines the base case for recursion. Here we simply project away the *z* attributes and assign an annotation value of *1/N* (where *N* is our scalar relation holding the number of nodes). Finally, the third line defines the recursive rule which joins the *Edge* and *InvDegree* relations inside the database with the new *PageRank* relation. We *SUM* over the *z* attribute in all of these relations. When aggregated attributes are joined with each other their annotation values are multiplied by default using the default definition of our semi-ring. Therefore we are performing a matrix-vector-vector multiplication. After the aggregation the corresponding expression for the annotation *y* is applied to each aggregated value. This is run for a fixed number (5) iterations as specified in the head.